In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import optuna
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.mlp.mlp_cv_trainer_emb as cv
import src.models.mlp.mlp_optuna_optimizer as op
import src.utils.telegram as te

In [2]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df8 = pd.read_parquet("../artifacts/features/base/tr_df8.parquet")
tr_df7 = pd.read_parquet("../artifacts/features/base/tr_df7.parquet")

In [ ]:
# Tuning
importlib.reload(cv)
importlib.reload(op)
objective = op.create_objective(
    tr_df8,
    early_stopping_rounds=10,
    min_epochs=10,
)

op.run_optuna_search(
    objective,
    n_trials=100,
    direction="minimize",
    study_name="mlp_v6",
    storage=url,
    sampler=optuna.samplers.TPESampler(n_startup_trials=40, seed=42),
)
te.send_telegram_message("MLP Training Complete!")

[I 2025-08-03 04:14:14,631] A new study created in RDB with name: mlp_v6


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1: Train Logloss = 0.18210, Val Logloss = 0.18229
New best model saved at epoch 1, Logloss: 0.18229
Epoch 2: Train Logloss = 0.17423, Val Logloss = 0.17433
New best model saved at epoch 2, Logloss: 0.17433
Epoch 3: Train Logloss = 0.16981, Val Logloss = 0.16993
New best model saved at epoch 3, Logloss: 0.16993
Epoch 4: Train Logloss = 0.16784, Val Logloss = 0.16761
New best model saved at epoch 4, Logloss: 0.16761
Epoch 5: Train Logloss = 0.16466, Val Logloss = 0.16429
New best model saved at epoch 5, Logloss: 0.16429
Epoch 6: Train Logloss = 0.16360, Val Logloss = 0.16341
New best model saved at epoch 6, Logloss: 0.16341
Epoch 7: Train Logloss = 0.16167, Val Logloss = 0.16109
New best model saved at epoch 7, Logloss: 0.16109
Epoch 8: Train Logloss = 0.16034, Val Logloss = 0.15990
New best model saved at epoch 8, Logloss: 0.15990
